<a href="https://colab.research.google.com/github/khadkechetan/information_extraction/blob/main/Docvqa/pix2struct/Pix2Struct_docVQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Chetan Khadke

Email: khadkechetan@gmail.com

Linkedin: https://www.linkedin.com/in/khadke-chetan/ ☕

# Install the packages

In [ ]:
!pip install git+https://github.com/huggingface/transformers pdf2image
!sudo apt install poppler-utils

### Load the packages

In [2]:
from pdf2image import convert_from_path, convert_from_bytes
import torch
from functools import partial
from PIL import Image
from transformers import Pix2StructForConditionalGeneration as psg
from transformers import Pix2StructProcessor as psp

## Specify questions and filename

In [3]:
questions = ["what is the seller name?",
             "what is the date of issue?",
             "What is Delivery address?",
             "What is Tax Id of client?"]
FILENAME = "/content/invoice_107_charspace_108.pdf"
PAGE_NO = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Load the Model and Processor

In [4]:
model = psg.from_pretrained("google/pix2struct-docvqa-large").to(DEVICE)
processor = psp.from_pretrained("google/pix2struct-docvqa-large")


## Helper Script

In [5]:
def generate(model, processor, img, questions):
  inputs = processor(images=[img for _ in range(len(questions))], text=questions, return_tensors="pt").to(DEVICE)
  predictions = model.generate(**inputs, max_new_tokens=256)
  return zip(questions, processor.batch_decode(predictions, skip_special_tokens=True))

def convert_pdf_to_image(filename, page_no):
    return convert_from_path(filename)[page_no-1]


## Generate Results

In [6]:
image = convert_pdf_to_image(FILENAME, PAGE_NO)
print("pdf to image conversion complete.")
generator = partial(generate, model, processor)
completions = generator(image, questions)
for completion in completions:
    print(f"{completion}")

pdf to image conversion complete.


('what is the seller name?', 'Campbell, Callahan and Gomez')
('what is the date of issue?', '09/25/2011')
('What is Delivery address?', '2969 Todd Orchard Apt. 721')
('What is Tax Id of client?', '941-79-6209')
